In [ ]:
import os 
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../")))
from gitstats import NWBGitInfo, GitHubRepoInfo

In [ ]:
import git
import requests
from datetime import datetime
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

In [ ]:
git_repo_infos = NWBGitInfo.GIT_REPOS.get_info_objects()

In [ ]:
# Plot results for select repos
select_repos = ['PyNWB', 'HDMF', 'MatNWB', 'NWB_Schema', 'HDMF_Common_Schema']
fig, axes = plt.subplots(figsize=(16, len(select_repos)*4.2), 
                         nrows=len(select_repos), 
                         ncols=1, sharex=True, sharey=False, squeeze=True)
for i, repo in enumerate(select_repos):
    git_repo_infos[repo].plot_release_timeline(
        fontsize=14, 
        month_intervals=2,
        xlim=(datetime.strptime("2017-10-01", "%Y-%m-%d"), 
             datetime.strptime("2021-10-01", "%Y-%m-%d"),),
        ax=axes[i],
        title_on_yaxis=True)
    axes[i].grid(axis="x", linestyle='dashed', color='lightgray')
axes[0].set_title("Timeline of NWB Release", fontdict={'fontsize':16})
plt.subplots_adjust(wspace=0, hspace=0.0)
plt.savefig('nwb_software_releases_timeline.pdf', dpi=300)
plt.show()